In [1]:
import os
os.chdir('D:\\Gnan\\DA\\KMU\\Prediction-COVID-19')
from utils import *
from model import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
key = 'Infected'
var_list = ['up1', 'up2', 'down1', 'down2']
cities = ['강원', '경기', '경남', '경북', '광주', '대구', 
          '대전', '부산', '서울', '세종', '울산', '인천',
          '전남', '전북', '제주', '충남', '충북']
models = ['RNN', 'LSTM', 'GRU', 'BiRNN', 'BiLSTM', 'BiGRU', 
          'seq2seq_RNN', 'seq2seq_LSTM', 'seq2seq_GRU', 
          'seq2seq_BiRNN', 'seq2seq_BiLSTM', 'seq2seq_BiGRU']

In [3]:
var_num = 0
dic_total_model_final = {}
dic_model_rank = {}
for name in models:
    dic_model_rank[name] = 0

for var in var_list:
    
    path = f'Data/Stricgency/{key}/{var}/*.csv'
    dic_files = Load_files.load_files(path, -17, -15)

    dic_hyperparameter = {}
    for city in cities:
        path = f'hyperparameter/Stricgency/{key}/{var}/{city}/*.pkl'
        files = glob.glob(path)
        hyperparameter = {}
        for file in files:
            if var_num < 2:
                hyperparameter[file[49:-4]] = load_hyperparameter(file)
            else:
                hyperparameter[file[53:-4]] = load_hyperparameter(file)
                
        dic_hyperparameter[city] = hyperparameter
        
    city_predicted = {}

    for city in cities:

        dic_model = load_model_multiple(dic_hyperparameter, 'Stricgency', key, var, city)
        data = dic_files[city]
        df = Prepare_df.processing(data, 'stdDay', 'defCnt')

        x = df.iloc[:, 0:]
        y = df.iloc[:,:1]

        ms = MinMaxScaler()
        ss = StandardScaler()

        ss.fit(x)
        ms.fit(y)

        predicted_values = {}
        num_model = 0
        for name, state in dic_model.items():
            batch_size = 64
            if num_model < 6:
                x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 138, 60, 1, batch_size, 'mto')
                label_y, predicted = predict_mto(state, df, x_ss, y_ms)
                predicted_values[name] = [label_y, predicted]
            else:
                x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 132, 60, 7, batch_size, 'mtm')
                label_y, predicted, first_label_y, first_predicted, all_predicted = predict_mtm(state, df, x_ss, y_ms, 7, 0.5, device)
                predicted_values[name] = [label_y, predicted, first_label_y, first_predicted, all_predicted]

            num_model += 1

        city_predicted[city] = predicted_values

    dic_best_model = {}
    dic_total_model = {}

    for city in cities:

        actual_data = city_predicted[city]['RNN'][0]
        rmse_min = np.inf
        num_model = 0
        dic_model = {}

        for name, file in city_predicted[city].items():
            if num_model < 6:
                predicted_data = file[1]
            else:
                predicted_data = file[4]

            rmse_val = rmse(actual_data[-7:], predicted_data[-7:])
            dic_model[name] = [rmse_val, actual_data, predicted_data]

            num_model += 1

        dic_total_model[city] = dic_model
    dic_total_model_final[var] = dic_total_model
    var_num += 1

    for city in cities:
        rmse_rank = list(dict(sorted(dic_total_model[city].items(), key = lambda x:x[1][0])).keys())
        for m in rmse_rank:
            dic_model_rank[m] += rmse_rank.index(m)
    
    print(rmse_rank)

['LSTM', 'GRU', 'BiGRU', 'RNN', 'seq2seq_GRU', 'BiLSTM', 'BiRNN', 'seq2seq_RNN', 'seq2seq_LSTM', 'seq2seq_BiGRU', 'seq2seq_BiRNN', 'seq2seq_BiLSTM']
['BiRNN', 'BiLSTM', 'RNN', 'BiGRU', 'GRU', 'seq2seq_RNN', 'seq2seq_LSTM', 'LSTM', 'seq2seq_GRU', 'seq2seq_BiGRU', 'seq2seq_BiLSTM', 'seq2seq_BiRNN']
['BiLSTM', 'seq2seq_BiGRU', 'BiRNN', 'BiGRU', 'RNN', 'seq2seq_GRU', 'GRU', 'seq2seq_RNN', 'seq2seq_LSTM', 'seq2seq_BiLSTM', 'seq2seq_BiRNN', 'LSTM']
['seq2seq_GRU', 'seq2seq_LSTM', 'seq2seq_BiGRU', 'seq2seq_BiRNN', 'seq2seq_RNN', 'seq2seq_BiLSTM', 'BiLSTM', 'BiRNN', 'RNN', 'BiGRU', 'LSTM', 'GRU']


In [4]:
model_rank_list = list(dict(sorted(dic_model_rank.items(), key = lambda x:x[1])).keys())
model_rank_list

['seq2seq_GRU',
 'BiGRU',
 'BiLSTM',
 'LSTM',
 'GRU',
 'seq2seq_RNN',
 'seq2seq_LSTM',
 'BiRNN',
 'RNN',
 'seq2seq_BiGRU',
 'seq2seq_BiRNN',
 'seq2seq_BiLSTM']

In [5]:
top_model = model_rank_list[:6]
top_model

['seq2seq_GRU', 'BiGRU', 'BiLSTM', 'LSTM', 'GRU', 'seq2seq_RNN']

In [6]:
dic_var_diff = {}

for var in var_list:
    path = f'Data/Stricgency/{key}/{var}/*.csv'
    dic_files = Load_files.load_files(path, -17, -15)
    dic_city_diff = {}
    for city in cities:
        data = dic_files[city]
        rmse_min = np.inf
        dic_model_diff = {}
        print(var, city)
        for name in top_model:
            rmse_val = rmse(dic_total_model_final[var][city][name][1][-7:], dic_total_model_final[var][city][name][2][-7:])
            diff_val = float(sum(dic_total_model_final[var][city][name][2][-7:] - dic_total_model_final[var][city][name][1][-7:]))
            dic_model_diff[name] = [rmse_val, diff_val]
            #plotting(dic_total_model[city][t][1], dic_total_model[city][t][2], 132)
            if rmse_min > rmse_val:
                rmse_min = rmse_val
                diff_min = diff_val
                best_model = name
                best_predicted = dic_total_model_final[var][city][name][2]
                
            df = pd.DataFrame(np.concatenate([dic_total_model_final[var][city][name][1]
                                              , dic_total_model_final[var][city][name][2]]
                                             , axis = 1), 
                              columns = ['Actual_data', 'Predicted_data'])
            df.index = list(data['stdDay'][-1*len(actual_data):])
            df.to_csv(f"Data/Predicted/Stricgency/{key}/{var}/{city}/{name}.csv", encoding = 'cp949')
        dic_model_diff['best_model'] = [best_model, rmse_min, diff_min]
        dic_city_diff[city] = dic_model_diff
        df_best = pd.DataFrame(np.concatenate([dic_total_model_final[var][city][name][1]
                                              , best_predicted]
                                             , axis = 1), 
                              columns = ['Actual_data', 'Predicted_data'])
        df_best.index = list(data['stdDay'][-1*len(actual_data):])
        df_best.to_csv(f"Data/Predicted/Stricgency/{key}/{var}/{city}/best_{best_model}.csv", encoding = 'cp949')
        print(rmse_min, best_model)
    dic_var_diff[var] = dic_city_diff
    
with open(f'Data/Predicted/Stricgency/{key}/valid_score.pkl', 'wb') as f:
    pickle.dump(dic_var_diff, f)

up1 강원
45.7804527447646 GRU
up1 경기
315.64016052698366 LSTM
up1 경남
32.23171265665618 seq2seq_GRU
up1 경북
7.245917650740889 LSTM
up1 광주
30.66937155473682 LSTM
up1 대구
50.12932493810384 GRU
up1 대전
11.374896038544499 seq2seq_GRU
up1 부산
81.02399155207827 GRU
up1 서울
176.73462848646838 seq2seq_GRU
up1 세종
0.8461430134613364 LSTM
up1 울산
54.419418818389175 GRU
up1 인천
47.04565290259293 GRU
up1 전남
16.047156460530633 seq2seq_GRU
up1 전북
17.360600274602582 LSTM
up1 제주
24.315494774583808 GRU
up1 충남
31.71414490893121 BiGRU
up1 충북
23.332679666718256 LSTM
up2 강원
55.446269652334365 GRU
up2 경기
285.0368123155323 GRU
up2 경남
322.0559078987684 seq2seq_RNN
up2 경북
53.681747181595625 BiGRU
up2 광주
42.308759470465304 BiGRU
up2 대구
167.9020652512202 BiGRU
up2 대전
165.5641279648161 seq2seq_GRU
up2 부산
114.42696906761098 BiLSTM
up2 서울
969.8139963415665 BiGRU
up2 세종
14.338517840839419 seq2seq_GRU
up2 울산
18.556375745563756 LSTM
up2 인천
109.72745443968205 BiGRU
up2 전남
41.54892870535292 GRU
up2 전북
142.642568484131 seq2seq_GRU
u

In [7]:
dic_var_diff

{'up1': {'강원': {'seq2seq_GRU': [75.68610328500702, -518.5836181640625],
   'BiGRU': [68.574260471541, -473.5709228515625],
   'BiLSTM': [81.75878261787078, -561.2244873046875],
   'LSTM': [118.24170408928273, -792.9986572265625],
   'GRU': [45.7804527447646, -317.7384033203125],
   'seq2seq_RNN': [83.40454346792956, -571.9320068359375],
   'best_model': ['GRU', 45.7804527447646, -317.7384033203125]},
  '경기': {'seq2seq_GRU': [731.3923365745638, -5009.9384765625],
   'BiGRU': [318.887580602475, -2003.90625],
   'BiLSTM': [714.4486073189589, -4662.9951171875],
   'LSTM': [315.64016052698366, -2097.083984375],
   'GRU': [437.3784116757479, -2961.6708984375],
   'seq2seq_RNN': [741.828231466018, -4769.5380859375],
   'best_model': ['LSTM', 315.64016052698366, -2097.083984375]},
  '경남': {'seq2seq_GRU': [32.23171265665618, -210.244384765625],
   'BiGRU': [38.75761486419072, -243.3892822265625],
   'BiLSTM': [72.93712845342384, -486.3927001953125],
   'LSTM': [45.85456214097595, -288.975219726